<a href="https://colab.research.google.com/github/kameda-yoshinari/DataAlgo2020/blob/master/DataAlgo2020_011(KnightTour).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6.2.1. 騎士の巡回 (Knight tour)

全解探索法とバックトラック法の実際例として，騎士の巡回問題を取り上げる．


**いつもの約束**  
１つのコードセルだけの実行は Ctrl + Enter．  
エディタで「インデント幅（スペース）は4で表示」「行番号を表示」「インデントガイドを表示」．  
内部では日本語はUTF-8で表現されている．


# 準備

インスタンスに接続し起動する．  
下記の手順でGoogle Driveをマウントする．  
マウント先に移動し，作業フォルダとする．  
これによって，インスタンスがリセットされてもGoogle Drive内にファイルが保存されるようにする．

In [ ]:
!echo "Google Driveをマウントします"
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
!echo "今回の作業用フォルダを作成しそこに移動します"
%cd /content/drive/My\ Drive/
%mkdir -p 202004_DataAlgo/DA2020_011
%cd       202004_DataAlgo/DA2020_011
!ls
!echo "日本時間表示"
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Japan /etc/localtime
!date

# 騎士の巡回

**問題**

チェスのナイトは移動先を8つの中から選べる．前後左右に2マス進んだ後，どちらかの直角方向に1マス進んだ位置である．騎士の巡回 (Knight tour) とは，

* チェスの8x8の盤の中で
* 同じマスにを2度通ることなしに（63手かけて）
* ナイトの移動方法のみで
* 塗りつぶす

ことができるかどうかを調べる問題である．

なお，最終移動の次の移動で開始マスに戻れる場合，騎士の周遊 (Knight tour round)という．騎士の周遊の探索だけならどこを開始マスに選んでも結果は同じである．



# 全解探索法

**解説**

全解探索 (bluteforce search / exhausive search) とは，一部の解の条件のみを満たす解候補を一旦生成し，その解候補が解の条件すべてを満たすかどうかを確認して解を見つけようとする方法である．

> 暗黙の前提になっているのは，全ての条件を上手に利用して解を直接求めるアルゴリズムを簡単には思いつけない，という状況である．さらに，解候補が解の条件をすべて満たすかどうかを調べること（十分条件の確認）は簡単である，ということも全解探索を試みるべきかどうかの判断材料となる．

> 解候補の十分条件の確認が並列実行できる場合，並列実行可能な計算資源を用意して全解探索を実際に用いることはある．

> 全解探索は解の探索の一手法であるだけでなく，その解候補の多さが問題の難しさを表す指標として使われることがある．

解の探索に全解探索を用いる場合は，解候補の多さに注意する．

全解探索のときの基本方針として，実際には，解があるかどうかを早く知りたいので，深さ優先探索（出来るだけ早く1つ目の解を得るよう努力する）が用いられることが多い．

**騎士の巡回**

全解探索として，「ナイトの移動方法のみで」「同じマスにを2度通ることなし（63手かける）」条件から解候補を生成する方法を考える．

最初にナイトが盤上に置いてあるとする．
残りは63マスである．
つまり，63回移動を繰り返して，それが丁度全てのマスを塗りつぶしていれば解であると言える．

1回の移動で選択肢は8あるので，考えられる解候補は 8<sup>63</sup> (8の63乗)通りあることになる．

解候補の十分条件の確認には，「8x8の盤の中で」「塗りつぶす」を確認する必要がある．これは解候補による移動状況を調べるだけで可能であり，高々8<sup>2</sup>のコストが必要である．

つまり，8<sup>63</sup> x 8<sup>2</sup> = 8<sup>65</sup> 程度が全解探索における時間計算量である．これだけの時間を費やせば，解があるかどうかの判定は必ず終了する．



# 全解探索法のCプログラム

**目標**

盤の横縦サイズ (boardsize.x と boardsize.y) と開始マスを指定して，ナイトで塗りつぶしができるかどうかを判定するプログラムを作成する．
起動時に，第1,第2引数で盤の横縦の大きさを，第3,第4引数で開始マスを指定する．

**説明**

XY座標については，vec2i構造体を導入する．
全解探索では，全マス数 - 1 の手まで全ての考えられる経路を挙げ，その一つ一つについて十分条件を確認する．全ての解候補を挙げていくために深さ優先探索を用いる．

**コード**

kpath[]配列で全手の移動の様子を記録する．一手ごとにナイトの8方向移動のうちの一つが選択されるので，それを記録していく．
kpath[]で表現する解候補は，残念ながらほとんどの場合は盤に収まらない．状況確認のため，盤に載った場合のみ表示する関数を用意する．盤上での手の表記は，初期位置(初手)を1とする．  
(マスの表記で0は未踏を示す)

**備考**

状態を標準出力で確認しやすいよう，出力の細かさを制御するverboselevel変数を用意する．起動時の第５引数で指定し，0以上(4以下)で値が大きい方が細かくなる．

In [ ]:
%%writefile knighttour-bluteforce_J.c
// Knight tour by bluteforce search
// kameda[at]ccs.tsukuba.ac.jp, 2020.
#include <stdio.h>  // printf
#include <stdlib.h> // atoi

#define M 8 // ナイトの移動選択肢数

// ２次元座標
typedef struct {int x; int y;} vec2i;

// ナイトの移動先（相対移動量）
vec2i move[M] = {
	{ 2 ,1},
	{ 1, 2},
	{-1, 2},
	{-2, 1},
	{-2,-1},
	{-1,-2},
	{ 1,-2},
	{ 2,-1}
};

// 経路の表現
int *kpath = NULL; // 各手のナイトの方向を記録

// 盤の表現
int *board = NULL; // (x,y)の値は board[y * boardsize.x + x] で参照
vec2i boardsize = {0, 0};

// 初期位置（騎士の周遊の判定用）
vec2i spt = {0, 0};

int knighttour      = 0; // 騎士の巡回 の解数
int knighttourround = 0; // 騎士の周遊 の解数
int verboselevel    = 0; // 表示レベル (0で最小)

// 盤の座標へのアクセス
int boardindex(vec2i pt){
	return (pt.y * boardsize.x + pt.x);
}

// 盤の整備と現在位置
//    s手目まで確定しているとする．
//    初期位置(初手)は spt で与えられているので，残りs-1手．
//    つまり kpath[0], ... , kpath[s-2] までを参照して盤を表現する．
vec2i board_from_path(int s){
    int c;
	vec2i pt;
 
    for (c = 0; c < boardsize.x * boardsize.y; c++) {
        board[c] = 0; // 未踏
    }
    board[boardindex(spt)] = 1; // 初手
    pt = spt;
    if (verboselevel >= 3) printf("** - (%d, %d) - at %d\n", pt.x, pt.y, s);
    for (c = 0; c < s - 1; c++) {
        pt.x += move[kpath[c]].x;
        pt.y += move[kpath[c]].y;
        if (verboselevel >= 3) printf(">> %d (%d, %d) %d at %d\n", c, pt.x, pt.y, kpath[c], s);
        if (pt.x >= 0 && pt.x < boardsize.x &&
            pt.y >= 0 && pt.y < boardsize.y ) {
            board[boardindex(pt)] = c + 2; // 移動先が盤内のときは表記
        }
    }
    return pt;
}
 
// 盤の表示(左右をx軸，上下をy軸とする)
void showboard(void){
    vec2i pt;
 	for (pt.y = 0; pt.y < boardsize.y; pt.y++) {
		for (pt.x = 0; pt.x < boardsize.x; pt.x++) {
			printf("%2d ", board[boardindex(pt)]);
		}
		printf("\n");
	}
	return ;
}

// 騎士の巡回の判定
int check_knighttour(void){
	int c;
    
    for (c = 0; c < boardsize.x * boardsize.y; c++) {
        if (board[c] <= 0)
            return 0;
    }
	return 1;
}

// 騎士の周遊の判定
int check_knighttourround(vec2i pt){
	int d;
	for (d = 0; d < M; d++) {
		if (pt.x + move[d].x == spt.x && 
		    pt.y + move[d].y == spt.y) {
			return 1;
		}
	}
	return 0;
}

// 解の探索	
//   int n_move; number of moves to examine here
void knighttouronestep_bf(int n_move){
    vec2i cpt;
	int d; // choice of direction

    if (verboselevel >= 2) printf("-- %d cells fixed --\n", n_move);

    cpt = board_from_path(n_move);
    if (verboselevel >= 2) showboard();

	// 解判定
	if (n_move == boardsize.x * boardsize.y) {
        // 騎士の巡回の解か？
        if (check_knighttour() > 0) {
            knighttour++;
            // 騎士の周遊の解か？
            knighttourround += check_knighttourround(cpt);
            // 解の表示
		    if (verboselevel >= 1) {
			    printf("Answer found (%d / %d).\n", knighttour, knighttourround);
	    		showboard();
	    	}
        }
        return ; // 終了せずに解の探索を継続
	}
	
	// 解探索
	for (d = 0; d < M; d++) {
		kpath[n_move - 1] = d; // １歩前進した分を記録
		knighttouronestep_bf(n_move + 1); // recursive call ; depth first
	}
	return ;
}

// メイン関数
//   盤のXサイズ 盤のYサイズ 開始X座標 開始Y座標
int main(int argc, char *argv[]){

	// 引数の確認
    if (argc != 6) {
        printf("盤のXサイズ, 盤のYサイズ, 開始X座標, 開始Y座標, 表示レベル の５つを指定してください．\n");
        printf("これらの指定がなかったので終了します．\n");
        return -1;
    }
    // 第1,2引数で盤の大きさ
	boardsize.x = atoi(argv[1]);
	boardsize.y = atoi(argv[2]);
    // 第3,4引数で開始マス
	spt.x = atoi(argv[3]);
	spt.y = atoi(argv[4]);
	verboselevel = atoi(argv[5]);
	printf("board_size = (%d, %d), start_point = (%d, %d), verbose level = %d\n", boardsize.x, boardsize.y, spt.x, spt.y, verboselevel);
	if (boardsize.x < 0 || boardsize.y < 0 || 
	    spt.x < 0 || spt.x >= boardsize.x || 
		spt.y < 0 || spt.y >= boardsize.y) {
		printf("引数が正しくないので終了します．\n");
		return -2;
	}
	if ((kpath = calloc(boardsize.x * boardsize.y, sizeof(int))) == NULL) {
		printf("経路用の%d要素のメモリ確保に失敗したの終了します．\n", boardsize.x * boardsize.y);
		return -2;
	}
	if ((board = calloc(boardsize.x * boardsize.y, sizeof(int))) == NULL) {
		printf("盤用の%d要素のメモリ確保に失敗したの終了します．\n", boardsize.x * boardsize.y);
		return -2;
	}
	knighttouronestep_bf(1); // 移動の一手目

	printf("Result:\n");
	printf("  Knight tour       %6d\n", knighttour);
	printf("  Knight tour round %6d\n", knighttourround);

	return 0;
}


コンパイルしてバグがないことを確認する．  
今回からは実行速度が問題になるので，最適化についても考慮すべきである．


In [ ]:
!gcc -Wall -o knighttour-bluteforce_J knighttour-bluteforce_J.c

実行においては，timeコマンドを用いて計測するとよい．  
コマンド実行の前に time とつけて実行すると，実行時間を計測できる．  
このうち，user属性がプログラム実行にかかった値である．

**注意**

実行確認している間は，盤のサイズを最小限の 3x3 程度にしておくこと．  
3x3 では当然ではあるが解はない．
おそらく，実行時間は1秒前後である(2020/06現在)．
全解探索では，盤のサイズを大きくするとすぐに指数爆発を起こすので十分に注意すること．  
例えば，3x3に対して，4x3の盤を実行する前に，どれぐらい時間がかかるか見積もってから実行すること．

In [ ]:
!time ./knighttour-bluteforce_J  4 3 0 0 1

第5引数のverboselevelを上げるとcolab notebookの出力セルでは扱いきれなくなる（ブラウザに負荷がかかって操作不能になる）．  
標準出力を外部ファイルに誘導し，実行終了後，google driveで確認する方法を試すこと．しかし，これですら猛烈にファイルサイズが大きくなることに注意．  
例えば，3x3の盤でverboselevel 2ですら，標準出力を保存するファイル (下記の bluteforce33-00-2.txt) のファイルサイズは 914MB程度になる！

In [ ]:
!time ./knighttour-bluteforce_J 3 3 0 0 1

In [ ]:
!time ./knighttour-bluteforce_J 3 3 0 0 2 > bluteforce33-00-2.txt

In [ ]:
!ls -l bluteforce33-00-2.txt

盤を4x3にしての実行例が下記であるが，3x3の実行時間を元に，実行時間を見積もってから実行すること．

In [ ]:
!time ./knighttour-bluteforce_J 4 3 0 0 1

さらに盤を4x4にしての実行例が下記である．実行前によくよく見積もりを行うこと．

In [ ]:
!time ./knighttour-bluteforce_J 4 4 0 0 1 > bluteforce44-00-1.txt

# バックトラック法

全解探索の方法において，解が満たすべき条件を考え，探索においてその先に条件を満たす解が得られないことが自明な場合，その時点で引き返す方法をバックトラック(backtrack) 法という．後戻り法とも呼ばれる．

バックトラックを行えば，実効上，計算時間を削減できることになる．  
一方で，計算量がビッグオーとして削減できているかどうかについては慎重な議論が必要である．（ビッグオーダー表現では最悪値を考えなければならないことに注意する．）

騎士の巡回における今回のアルゴリズムでは，

* 盤の外に出る
* 以前に訪れたことがあるマスをもう一度訪れる

のどちらかに合致した時点でバックトラックする（深さ優先探索において後戻りする）．

ここで注意すべきは，バックトラック法を行っても，解の探索結果は全解探索と同一であるということである．


# バックトラック法によるCプログラム

**目標**

目標は全解探索によるCプログラムと同じであるが，バックトラック法により高速化を図る．

**説明**

XY座標については，vec2i構造体を導入する．
バックトラック法では，２条件とも問題ない場合のみ先に進む．言い換えれば，移動先が盤内であってかつ新しいマスであれば先に進む．
もっとも重要な点は，バックトラックする際に，一旦書いた記録を消去することである．すなわち，本プログラムでは探索を先に進めるときに「１歩前進した分を記録」し，戻るときに「１歩前進した分を消去」する．

**コード**

２次元に広がる盤そのものをboard[]配列で表現し，手番号(1からN<sup>2</sup>まで)を記入する．値0は未踏を表す．

**備考**

状態を標準出力で確認しやすいよう，出力の細かさを制御するverboselevel変数を用意する．起動時の第５引数で指定し，0以上(2以下)で値が大きい方が細かくなる．


In [ ]:
%%writefile knighttour-backtrack_J.c
// Knight tour by backtrack search
// kameda[at]ccs.tsukuba.ac.jp, 2020.
#include <stdio.h>  // printf
#include <stdlib.h> // atoi

#define M 8 // ナイトの移動選択肢数

// ２次元座標
typedef struct {int x; int y;} vec2i;

// ナイトの移動先（相対移動量）
vec2i move[M] = {
	{ 2 ,1},
	{ 1, 2},
	{-1, 2},
	{-2, 1},
	{-2,-1},
	{-1,-2},
	{ 1,-2},
	{ 2,-1}
};

// 盤の表現
int *board = NULL; // (x,y)の値は board[y * boardsize.x + x] で参照
vec2i boardsize = {0, 0};

// 初期位置（騎士の周遊の判定用）
vec2i spt = {0, 0};

int knighttour      = 0; // 騎士の巡回 の解数 
int knighttourround = 0; // 騎士の周遊 の解数
int verboselevel    = 0; // 表示レベル (0で最小)

// 盤の座標へのアクセス
int boardindex(vec2i pt){
	  return (pt.y * boardsize.x + pt.x);
}

// 盤の表示(左右をx軸，上下をy軸とする)
void showboard(void){
	vec2i pt;
	for (pt.y = 0; pt.y < boardsize.y; pt.y++) {
		for (pt.x = 0; pt.x < boardsize.x; pt.x++) {
			printf("%2d ", board[boardindex(pt)]);
		}
		printf("\n");
	}
	return ;
}

// 騎士の周遊の判定
int check_knighttourround(vec2i pt){
	int d;
	for (d = 0; d < M; d++) {
		if (pt.x + move[d].x == spt.x && 
		    pt.y + move[d].y == spt.y) {
			return 1;
		}
	}
	return 0;
}

// 解の探索 (backtrack)
//   vec2i cpt; current point
//   int n_move; number of fixed moves
void knighttouronestep_bt(vec2i cpt, int n_move){
	vec2i npt; // next point
	int d; // choice of direction

    if (verboselevel >= 2) printf("-- %d cells fixed --\n", n_move);
	if (verboselevel >= 2) showboard();

	// 解判定
	if (n_move == boardsize.x * boardsize.y) { 
	    // 騎士の巡回の解（確定）
		knighttour++; 
		// 騎士の周遊の解か？
		knighttourround += check_knighttourround(cpt);
		// 解の表示
		if (verboselevel >= 1) {
			printf("Answer found (%d / %d).\n", knighttour, knighttourround);
			showboard();
		}
		return ; // 終了せずに解の探索を継続
	}
	
	// 解探索
	for (d = 0; d < M; d++) {
		// 移動先座標
		npt.x = cpt.x + move[d].x;
		npt.y = cpt.y + move[d].y;
		if (npt.x >= 0 && npt.x < boardsize.x &&
		    npt.y >= 0 && npt.y < boardsize.y &&
			board[boardindex(npt)] == 0) {
			board[boardindex(npt)] = n_move + 1;   // １歩前進した分を記録
			knighttouronestep_bt(npt, n_move + 1); // recursive call ; depth first
			// Backtrack
			board[boardindex(npt)] = 0;            // １歩前進した分を抹消(原状復帰)
		}
	}
	return ;
}

// メイン関数
//   盤のXサイズ 盤のYサイズ 開始X座標 開始Y座標
int main(int argc, char *argv[]){

	// 引数の確認
    if (argc != 6) {
        printf("盤のXサイズ, 盤のYサイズ, 開始X座標, 開始Y座標, 表示レベル の５つを指定してください．\n");
        printf("これらの指定がなかったので終了します．\n");
        return -1;
    }
	// 第1,2引数で盤の大きさ
	boardsize.x = atoi(argv[1]);
	boardsize.y = atoi(argv[2]);
	// 第3,4引数で開始マス
	spt.x = atoi(argv[3]);
	spt.y = atoi(argv[4]);
	verboselevel = atoi(argv[5]);
	printf("board_size = (%d, %d), start_point = (%d, %d), verbose level = %d\n", boardsize.x, boardsize.y, spt.x, spt.y, verboselevel);
	if (boardsize.x < 0 || boardsize.y < 0 || 
	    spt.x < 0 || spt.x >= boardsize.x || 
		spt.y < 0 || spt.y >= boardsize.y) {
		printf("引数が正しくないので終了します．\n");
	}
	if ((board = calloc(boardsize.x * boardsize.y, sizeof(int))) == NULL) {
		printf("盤用の%d要素のメモリ確保に失敗したの終了します．\n", boardsize.x * boardsize.y);
		return -2;
	}
	board[boardindex(spt)] = 1; // 初手
	knighttouronestep_bt(spt, 1);

	printf("Result:\n");
	printf("  Knight tour       %6d\n", knighttour);
	printf("  Knight tour round %6d\n", knighttourround);

	return 0;
}



コンパイルしてエラーが無いことを確認．

In [ ]:
!gcc -Wall -o knighttour-backtrack_J knighttour-backtrack_J.c

実行．最初のうちは全解探索で実施した小さい盤のサイズから試すこと．  
全解探索での実行速度からすれば，実行速度が劇的に改善される様子を確認されたい．  
(しかしそれでも8x8は無茶である)

In [ ]:
!time ./knighttour-backtrack_J 6 6 0 0 0

# 大きい数

指数時間爆発の危険性がある問題についてプログラムを書く場合には，実行時間の他にも様々な限界に留意しなくてはいけない．
例えば，解の数を表現するために上記プログラムではint型を利用しているが，これは適切ではない．

8x8の盤で全解探索したときの解候補の数は8<sup>63</sup>であるが，これは2<sup>189</sup>である．つまり，8バイトのunsigned long型を使っても数え上げができない．桁あふれが途中で発生し得る．（解の数が64ビットで表現できる数字内に収まればよいが，そのこと自体はどうやって調べればよいのか？）

In [ ]:
%%writefile biginteger.c
#include <stdio.h>
#include <limits.h>
int main(int argc, char *argv[]){
    printf("整数型 int           の最大値 INT_MAX   %d\n", INT_MAX);
    printf("整数型 long          の最大値 LONG_MAX  %ld\n", LONG_MAX);
    printf("整数型 unsigned long の最大値 ULONG_MAX %lu\n", ULONG_MAX);
    printf("整数型 unsigned long のビット数は %ld\n", sizeof(unsigned long) * 8);
    return 0;
}


In [ ]:
!gcc -o biginteger biginteger.c
!./biginteger

# 節末課題

1. 全解探索法の計算量  
knighttour-bluteforce_J プログラムの時間計算量と空間計算量を議論せよ．盤のマスの数をNとしてよい．


2. バックトラック法の計算量  
knighttour-backtrack_J プログラムの時間計算量と空間計算量を議論せよ．盤のマスの数をNとしてよい．

3. バックトラック法での解候補の必要条件の実装  
knighttour-backtrack_J プログラムにおいて，「チェスの8x8の盤の中で」「同じマスにを2度通ることなしに（63手かけて）」「ナイトの移動方法のみで」「塗りつぶす」ことを確認している実装部分をそれぞれ示せ．

4. 解候補の十分条件の確認  
knighttour-bluteforce_J プログラムと knighttour-backtrack_J プログラムのそれぞれにおいて，解候補が騎士の巡回問題の十分条件を満たしていることを確認している部分を抜き出してその内容を説明せよ．

5. バックトラック時の記録抹消の意味  
knighttour-backtrack_J プログラムにおいて，L97の再帰呼出によってその先の深さ優先探索を終えたあと，なぜL99のように記録を抹消しなくてはいけないのか，理由を説明せよ．

6. verboselevelの説明  
knighttour-bluteforce_J プログラムと knighttour-backtrack_J プログラムのそれぞれにおいて，verboselevelを0以上の値にすることで，それぞれ何が表示されることになるのか説明せよ．

7. 実行速度の改善  
knighttour-bluteforce_J プログラムと knighttour-backtrack_J プログラムでは実行速度が問題になる．そこで，gccのコンパイルオプションについて調べ，実際にそれが実行速度にどのような違いをもたらすかを調査せよ．実行時間が1分以上かかるような事例で調査すること．

8. 大きい数への対処  
大きい数に関する議論で述べたように，2<sup>189</sup>まで整数で数え上げる必要があるとき，Cプログラムでの実装にどのような方法が考えられるか説明せよ．




# 出典

筑波大学工学システム学類  
データ構造とアルゴリズム  
担当：亀田能成  
2020/06/24.